In [ ]:
#import
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
#解凍
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z
!p7zip -x /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z


In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
#ファイルの場所確認
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_table("/kaggle/working/train.tsv")
train_data

In [ ]:
#欠損値書き換え
train_data = train_data[train_data['price'] > 0].reset_index(drop=True)
train_data

In [ ]:
#数値化
train_data.category_name = train_data.category_name.astype('category')
train_data.item_description = train_data.item_description.astype('category')
train_data.name = train_data.name.astype('category')
train_data.brand_name = train_data.brand_name.astype('category')

In [ ]:
train_data.dtypes

In [ ]:
train_data.name = train_data.name.cat.codes
train_data.category_name = train_data.category_name.cat.codes
train_data.brand_name = train_data.brand_name.cat.codes
train_data.item_description = train_data.item_description.cat.codes

In [ ]:
#トレインデータ
train_data.head()
train_data.dtypes

In [ ]:
type(train_data)

In [ ]:
#train_np = np.array(train_data)
#train_np

In [ ]:
x = train_data.drop("train_id", axis=1)
x_train_data = x.drop("price", axis=1)
x_train_data

In [ ]:
y_train_data = train_data["price"]
y_train_data

In [ ]:
x_train_tensor = torch.tensor(np.array(x_train_data)).float()
y_train_tensor = torch.tensor(np.array(y_train_data)).float()
y_train_tensor = y_train_tensor.unsqueeze(1)

In [ ]:
x_train_tensor

In [ ]:
dataset_train = TensorDataset(x_train_tensor, y_train_tensor)

In [ ]:
train_loader = DataLoader(dataset=dataset_train, batch_size=50, shuffle=True)

In [ ]:
#ネットワークモデルの定義
model = nn.Sequential()
model.add_module('fc1', nn.Linear(6, 128))
model.add_module('relu', nn.ReLU())
model.add_module('fc2', nn.Linear(128, 64))
model.add_module('relu', nn.ReLU())
model.add_module('fc3', nn.Linear(64, 32))
model.add_module('relu', nn.ReLU())
model.add_module('fc4', nn.Linear(32, 1))

In [ ]:
#ハイパーパラメータ
loss_function = nn.L1Loss()
optimizer= optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
#学習
epoch  = 10
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
for i in range(epoch):
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_function(output, labels)
        optimizer.step()
    print("epoch:", i)

In [ ]:
#テストデータ
test_data = pd.read_table("/kaggle/working/test.tsv")
test_data

In [ ]:
test_data.category_name = test_data.category_name.astype('category')
test_data.item_description = test_data.item_description.astype('category')
test_data.name = test_data.name.astype('category')
test_data.brand_name = test_data.brand_name.astype('category')

In [ ]:
test_data.name = test_data.name.cat.codes
test_data.category_name = test_data.category_name.cat.codes
test_data.brand_name = test_data.brand_name.cat.codes
test_data.item_description = test_data.item_description.cat.codes

In [ ]:
x_test_data = test_data.drop("test_id", axis=1)
x_test_data

In [ ]:
#test_np = np.array(x_test_data)
#test_np

In [ ]:
dataset_test = torch.tensor(np.array(x_test_data)).float()
dataset_test

In [ ]:
test_loader = DataLoader(dataset=dataset_test, shuffle=False)

In [ ]:
out_price = pd.DataFrame(np.arange(test_np.shape[0]*2).reshape(test_np.shape[0], 2), columns=["test_id", "price"])
price_np = np.array(out_price)
price_np

In [ ]:
#結果
model.eval()
count = 0
for data in test_loader:
    output = model(data)
    price_np[count][0] = count
    price_np[count][1] = float(output)
    if count % 10000 == 0:
        print(count)
    if count < 10:
        print(output)
    count += 1

In [ ]:
#結果の書き込み，表示
price_np

In [ ]:
out_price = price_np
out_price = pd.DataFrame(price_np)
out_price.columns = ["test_id", "price"]

In [ ]:
out_price

In [ ]:
out_price.to_csv("submission.csv", index=False)